In [2]:
import numpy as np
import pickle
import os
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd

#import tensorflow as tf
from keras.models import Model
from keras import layers
from keras import Input
from keras.utils import plot_model

#from tensorflow.keras import layers
from keras.layers import Conv1D, Conv2D, Conv3D, GRU, GlobalAveragePooling1D, Activation, Flatten, Dropout, Dense, MaxPool1D, MaxPool2D, Concatenate, ZeroPadding2D, ZeroPadding1D, Reshape
# import BatchNormalization
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras import backend

from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images

from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


We will first process the Video data for subject 01 to use in the multi-modal

In [15]:
import xlrd
d = {}
wb = xlrd.open_workbook('/content/drive/My Drive/Deap/participant_response_valence.xls')
sh = wb.sheet_by_index(0)   
for i in range(881):
    cell_value_class = sh.cell(i,3).value
    cell_value_id = sh.cell(i,4).value
    d[cell_value_class] = cell_value_id
#d.get("s02_trail14")
#d["s02_trail14"]

In [16]:
# Break the video into frames

import os

def list_files(dir):
    Image_ID = []
    Class = []
    Image_ID_Frame = []
    Class_Frame = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            Image_ID.append(os.path.join(root, name))
            Class.append(d[name[0:11]])

            #read frames
            count = 0
            videoFile = os.path.join(root, name)
            cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
            frameRate = cap.get(5) #frame rate
            x=1
            while(cap.isOpened()):
                frameId = cap.get(1) #current frame number
                ret, frame = cap.read()
                if (ret != True):
                    break
                if (frameId % math.floor(frameRate) == 0):
                    #filename ="/content/drive/My Drive/Deap/DeapFrames/s01/frame%d.jpg" % count;count+=1
                    #path= videoFile.replace('DeapVideos','DeapFrames')
                    filename ="/content/drive/My Drive/Deap/DeapFrames/"+name[0:11]+"_frame%d.jpg" % count;count+=1
                    
                    #cv2.imwrite(filename, frame)
                    Image_ID_Frame.append(filename)
                    Class_Frame.append(d[name[0:11]])
                    print(filename)
                    print(d[name[0:11]])
            cap.release()

    # create dataframe from your lists
    df = pd.DataFrame(list(zip(Image_ID , Class)), 
    columns =['Image_ID', 'Class']) 
    data= pd.DataFrame(list(zip(Image_ID_Frame , Class_Frame)), 
    columns =['Image_ID_Frame', 'Class_Frame']) 
    return data

In [17]:
data_s01=list_files('/content/drive/My Drive/Deap/DeapVideos/s01')

/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame0.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame1.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame2.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame3.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame4.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame5.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame6.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame7.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame8.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame9.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame10.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame11.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame12.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame13.jpg
1.0
/content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame1

In [18]:
data_s01['Class_Frame'] = data_s01['Class_Frame'].astype(int)
data_s01['Image_ID_Frame'].shape

# prepare the data to use in model


X = [ ]     # creating an empty array
for img_name in data_s01.Image_ID_Frame:
    try:
      img = plt.imread('' + img_name)
      X.append(img)  # storing each image in array X
    except IOError:
      print("File "+ img_name+ "  not accessible")
      pass
X = np.array(X)    # converting list to array

y = data_s01.Class_Frame
y = np_utils.to_categorical(y)    # one hot encoding Classes

File /content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame0.jpg  not accessible
File /content/drive/My Drive/Deap/DeapFrames/s01_trial05_frame1.jpg  not accessible


In [19]:
X.shape

(2398, 576, 720, 3)

In [21]:
# removing the output label for the 2 frames that were not read in the beginning
y=y[2:2400]

In [22]:
y.shape

(2398, 2)

In [25]:
X.shape

(2398, 576, 720, 3)

In [26]:
y.shape

(2398, 2)

In [27]:
# Confirm the data was available for trail 5 frame 2
X[0]

array([[[186, 197, 163],
        [185, 196, 166],
        [182, 195, 169],
        ...,
        [104, 118, 119],
        [108, 122, 123],
        [141, 155, 156]],

       [[185, 196, 162],
        [185, 196, 166],
        [182, 195, 169],
        ...,
        [105, 119, 120],
        [108, 122, 123],
        [142, 156, 157]],

       [[186, 196, 162],
        [185, 196, 166],
        [182, 195, 169],
        ...,
        [105, 119, 120],
        [108, 122, 123],
        [142, 156, 157]],

       ...,

       [[ 29,  30,  25],
        [ 29,  30,  25],
        [ 29,  30,  25],
        ...,
        [241, 244, 235],
        [211, 216, 212],
        [250, 255, 255]],

       [[ 29,  30,  25],
        [ 29,  30,  25],
        [ 29,  30,  25],
        ...,
        [236, 239, 230],
        [199, 203, 202],
        [251, 255, 255]],

       [[ 29,  30,  25],
        [ 30,  31,  26],
        [ 29,  30,  25],
        ...,
        [231, 234, 225],
        [190, 194, 193],
        [251, 255, 255]]

In [28]:

##We will be using a VGG16 pretrained model which takes an input image of shape (224 X 224 X 3). Since our images are in a different size, we need to reshape all of them. We will use the resize() function of skimage.transform to do this.
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

#before passing any input to the model, we must preprocess it as per the model’s requirement. Otherwise, the model will not perform well enough. Use the preprocess_input() function of keras.applications.vgg16 to perform this step.
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X, mode='tf')      # preprocessing the input data



In [29]:
''' 
# saving data in case connection breaks
with open('/content/drive/My Drive/Deap/Pickle_Images/X_preprocess_s01.pkl', "wb") as fp:
  pickle.dump( X,fp, protocol=4)

with open('/content/drive/My Drive/Deap/Pickle_Images/y_preprocess_s01.pkl', "wb") as fp:
  pickle.dump( y,fp, protocol=4)
'''

In [30]:
# Validate the shape
X.shape

(2398, 224, 224, 3)

In [31]:
y.shape[0]

2398

Next we will load EEG spatial data for 24 channels that was earlier used in the CNN model 

In [32]:
arousal_or_valence = 'valence'
with_or_without = 'yes'
data_file_dir    = ['s01']
dataset_dir = "/content/drive/My Drive/Deap/deap_unshuffled_data/"+with_or_without+"_"+arousal_or_valence+"/"
cnn_suffix        =".mat_win_128_cnn_dataset.pkl"
#rnn_suffix        =".mat_win_128_rnn_dataset.pkl"
label_suffix    =".mat_win_128_labels.pkl"

window_size = 128

for data_file in data_file_dir:

  ###load training set
  with open(dataset_dir + data_file + cnn_suffix, "rb") as fp:
      cnn_datasets = pickle.load(fp)

  with open(dataset_dir + data_file + label_suffix, "rb") as fp:
      labels = pickle.load(fp)
      labels = np.transpose(labels)
      print("loaded shape:",labels.shape)
  labels_backup = labels
  print("cnn_dataset shape before reshape:", np.shape(cnn_datasets))
  cnn_datasets = cnn_datasets.reshape(len(cnn_datasets), window_size, 9,9, 1)
  print("cnn_dataset shape after reshape:", np.shape(cnn_datasets))
  one_hot_labels = np.array(list(pd.get_dummies(labels)))

  labels = np.asarray(pd.get_dummies(labels), dtype=np.int8)

loaded shape: (2400,)
cnn_dataset shape before reshape: (2400, 128, 9, 9)
cnn_dataset shape after reshape: (2400, 128, 9, 9, 1)


In [33]:
# Since we do not have 2 records in the video data and 2 frame were not read we want to delete the corresponding records in the EEG data

cnn_datasets = np.delete(cnn_datasets, (1140,1141), axis=0)
labels =np.delete(labels, (1140,1141), axis=0)

In [34]:
cnn_datasets.shape

(2398, 128, 9, 9, 1)

In [35]:
# Now we want to ensure that we are reading the video data in the same order in which we have the CNN EEG data. EEG data was available in the experiment order vs Video data was read at a random order of trials as we can see in the output from Deap_Video_V2.ipynb file. From the participant_rating.xls file we have the the ordering of trials compared to experiments

X_sorted = np.empty((2398, 224, 224, 3))
y_sorted = np.empty((2398,2))

X_sorted[0:60] = X[1258:1318]		
X_sorted[60:120] = X[2038:2098]		
X_sorted[120:180] = X[1798:1858]		
X_sorted[180:240] = X[178:238]		
X_sorted[240:300] = X[118:178]		
X_sorted[300:360] = X[478:538]		
X_sorted[360:420] = X[1438:1498]		
X_sorted[420:480] = X[1858:1918]		
X_sorted[480:540] = X[2278:2338]		
X_sorted[540:600] = X[718:778]		
X_sorted[600:660] = X[1198:1258]		
X_sorted[660:720] = X[1558:1618]		
X_sorted[720:780] = X[538:598]		
X_sorted[780:840] = X[1018:1078]		
X_sorted[840:900] = X[838:898]		
X_sorted[900:960] = X[1738:1798]		
X_sorted[960:1020] = X[898:958]		
X_sorted[1020:1080] = X[58:118]		
X_sorted[1080:1140] = X[2098:2158]		
X_sorted[1140:1198] = X[0:58]		
X_sorted[1198:1258] = X[2158:2218]		
X_sorted[1258:1318] = X[778:838]		
X_sorted[1318:1378] = X[2218:2278]		
X_sorted[1378:1438] = X[238:298]		
X_sorted[1438:1498] = X[1618:1678]		
X_sorted[1498:1558] = X[2338:2398]		
X_sorted[1558:1618] = X[1138:1198]		
X_sorted[1618:1678] = X[1678:1738]		
X_sorted[1678:1738] = X[1378:1438]		
X_sorted[1738:1798] = X[1318:1378]		
X_sorted[1798:1858] = X[358:418]		
X_sorted[1858:1918] = X[1498:1558]		
X_sorted[1918:1978] = X[598:658]		
X_sorted[1978:2038] = X[958:1018]		
X_sorted[2038:2098] = X[658:718]		
X_sorted[2098:2158] = X[1078:1138]		
X_sorted[2158:2218] = X[1918:1978]		
X_sorted[2218:2278] = X[1978:2038]		
X_sorted[2278:2338] = X[418:478]		
X_sorted[2338:2398] = X[298:358]		
		
		
y_sorted[0:60] = y[1258:1318]		
y_sorted[60:120] = y[2038:2098]		
y_sorted[120:180] = y[1798:1858]		
y_sorted[180:240] = y[178:238]		
y_sorted[240:300] = y[118:178]		
y_sorted[300:360] = y[478:538]		
y_sorted[360:420] = y[1438:1498]		
y_sorted[420:480] = y[1858:1918]		
y_sorted[480:540] = y[2278:2338]		
y_sorted[540:600] = y[718:778]		
y_sorted[600:660] = y[1198:1258]		
y_sorted[660:720] = y[1558:1618]		
y_sorted[720:780] = y[538:598]		
y_sorted[780:840] = y[1018:1078]		
y_sorted[840:900] = y[838:898]		
y_sorted[900:960] = y[1738:1798]		
y_sorted[960:1020] = y[898:958]		
y_sorted[1020:1080] = y[58:118]		
y_sorted[1080:1140] = y[2098:2158]		
y_sorted[1140:1198] = y[0:58]		
y_sorted[1198:1258] = y[2158:2218]		
y_sorted[1258:1318] = y[778:838]		
y_sorted[1318:1378] = y[2218:2278]		
y_sorted[1378:1438] = y[238:298]		
y_sorted[1438:1498] = y[1618:1678]		
y_sorted[1498:1558] = y[2338:2398]		
y_sorted[1558:1618] = y[1138:1198]		
y_sorted[1618:1678] = y[1678:1738]		
y_sorted[1678:1738] = y[1378:1438]		
y_sorted[1738:1798] = y[1318:1378]		
y_sorted[1798:1858] = y[358:418]		
y_sorted[1858:1918] = y[1498:1558]		
y_sorted[1918:1978] = y[598:658]		
y_sorted[1978:2038] = y[958:1018]		
y_sorted[2038:2098] = y[658:718]		
y_sorted[2098:2158] = y[1078:1138]		
y_sorted[2158:2218] = y[1918:1978]		
y_sorted[2218:2278] = y[1978:2038]		
y_sorted[2278:2338] = y[418:478]		
y_sorted[2338:2398] = y[298:358]		


In [36]:
# Confirm that the labels between the two datasets match

comparison = labels == y_sorted
equal_arrays = comparison.all()

print(equal_arrays)

True


In [37]:
# We are now ready to shuffle the data

index = np.array(range(0, len(labels)))
np.random.shuffle(index)

# randomly shuffled data
cnn_datasets   = cnn_datasets[index]
labels  = labels[index]

X_sorted = X_sorted[index]

# We can use just labels or y_sorted at this point as they have the same data

y_sorted=y_sorted[index]



In [ ]:
#video_y=np.asarray(y, dtype=np.int8)

In [39]:
# split the shuffled data for training, validation and testing
train_index = [i for i in range(1353)]
valid_index = [i for i in range(1353,1933)]
test_index  = [i for i in range(1933,2398)]

train_index = np.array(train_index)
valid_index = np.array(valid_index)
test_index = np.array(test_index)

video_data_train = X_sorted[train_index]
video_data_valid = X_sorted[valid_index]
video_data_test = X_sorted[test_index]
video_y_train = y_sorted[train_index]
video_y_valid = y_sorted[valid_index]
video_y_test = y_sorted[test_index]


cnn_model_train_x = cnn_datasets[train_index]
cnn_model_valid_x = cnn_datasets[valid_index]
cnn_model_test_x = cnn_datasets[test_index]
cnn_model_train_y = labels[train_index]
cnn_model_valid_y = labels[valid_index]
cnn_model_test_y = labels[test_index]

In [40]:
#Load the VGG model
#Video Model
vgg_conv_C = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

#mergedModel = Concatenate()([vgg_conv_C.output,vgg_conv_D.output])


print ("predict")

#We will make predictions using this model for X_train and X_valid, get the features, and then use those features to retrain the model.
video_data_train = vgg_conv_C.predict(video_data_train)
video_data_valid = vgg_conv_C.predict(video_data_valid)
video_data_test = vgg_conv_C .predict(video_data_test)
video_data_train.shape, video_data_valid.shape, video_data_test.shape

58892288/58889256 [==============================] - 5s 0us/step
predict


((1353, 7, 7, 512), (580, 7, 7, 512), (465, 7, 7, 512))

In [41]:
cnn_model_train_x.shape,cnn_model_train_y.shape, cnn_model_valid_x.shape,cnn_model_valid_y.shape

((1353, 128, 9, 9, 1), (1353, 2), (580, 128, 9, 9, 1), (580, 2))

In [44]:
ccn_in= Input(name='cnn_model_train_x', shape=(128,9, 9, 1))
video_in = Input(name='X_train', shape=(7*7*512,))  
#model.add(Dense(units=1024, activation='sigmoid')) 
V1= Dense(1024, activation='relu')(video_in)

C1 = Conv3D(32,(4,4,1), activation = 'relu')(ccn_in)
C2 = Conv3D(64,(4,4,1), activation = 'relu')(C1)
#C3 = Reshape(-1,9,192,1), activation = 'relu')(C2)
#F1= Reshape([-1,9,192])(C2)
#C3= Conv2D(64,(2,3), activation = 'relu')(F1)
F2=Flatten()(C2)
mergedModel = Concatenate()([F2,V1])

hidden = Dense(64, activation='relu')(mergedModel)
out = Dense(2, activation='softmax')(hidden)

model=Model([ccn_in,video_in],out)

model.compile(optimizer = "Adam",
               loss = 'binary_crossentropy',
               metrics = ['accuracy'])
# plot graph
plot_model(model)

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cnn_model_train_x (InputLayer)  (None, 128, 9, 9, 1) 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 125, 6, 9, 32 544         cnn_model_train_x[0][0]          
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 122, 3, 9, 64 32832       conv3d_1[0][0]                   
__________________________________________________________________________________________________
X_train (InputLayer)            (None, 25088)        0                                            
____________________________________________________________________________________________

In [45]:
# check if we need this reshaping

#The shape of X_train and X_valid is (1353, 7, 7, 512), (580, 7, 7, 512) respectively. In order to pass it to our neural network, we have to reshape it to 1-D.

video_data_train = video_data_train.reshape(video_data_train.shape[0], video_data_train.shape[1]*video_data_train.shape[2]*video_data_train.shape[3])      # converting to 1-D
video_data_valid = video_data_valid.reshape(video_data_valid.shape[0], video_data_valid.shape[1]*video_data_valid.shape[2]*video_data_valid.shape[3])
video_data_test= video_data_test.reshape(video_data_test.shape[0], video_data_test.shape[1]*video_data_test.shape[2]*video_data_test.shape[3])

# We will now preprocess the images and make them zero-centered which helps the model to converge faster.

video_data_train = video_data_train/video_data_train.max()      # centering the data
video_data_valid = video_data_valid/video_data_valid.max()
video_data_test = video_data_test/video_data_test.max() 

In [46]:
history = model.fit([cnn_model_train_x,video_data_train], video_y_train,
         
          epochs = 50, 
          batch_size = 32,  
          validation_data=([cnn_model_valid_x,video_data_valid],video_y_valid))

# add early stopping

history = model.fit([cnn_model_train_x,video_data_train], video_y_train,
          epochs = 50, 
          batch_size = 32,  
          validation_data=([cnn_model_valid_x,video_data_valid],video_y_valid),
          verbose = 1,
          callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights = True)])


Train on 1353 samples, validate on 580 samples
Epoch 1/50
1353/1353 [==============================] - 32s 23ms/step - loss: 0.9292 - accuracy: 0.5144 - val_loss: 0.6963 - val_accuracy: 0.5345
Epoch 2/50
1353/1353 [==============================] - 30s 23ms/step - loss: 0.6571 - accuracy: 0.5965 - val_loss: 0.5859 - val_accuracy: 0.7121
Epoch 3/50
1353/1353 [==============================] - 30s 22ms/step - loss: 0.3337 - accuracy: 0.8751 - val_loss: 0.1244 - val_accuracy: 0.9569
Epoch 4/50
1353/1353 [==============================] - 30s 22ms/step - loss: 0.0724 - accuracy: 0.9771 - val_loss: 0.0617 - val_accuracy: 0.9776
Epoch 5/50
1353/1353 [==============================] - 30s 22ms/step - loss: 0.0139 - accuracy: 0.9993 - val_loss: 0.0216 - val_accuracy: 0.9948
Epoch 6/50
1353/1353 [==============================] - 30s 22ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0210 - val_accuracy: 0.9931
Epoch 7/50
1353/1353 [==============================] - 30s 22ms/step - loss:

KeyboardInterrupt: ignored

In [47]:
test_loss, test_acc = model.evaluate([cnn_model_test_x,video_data_test], video_y_test) #test_y
print('test_acc:', test_acc)

465/465 [==============================] - 2s 5ms/step
test_acc: 0.9827957153320312
